# *Elements of Programming Interviews* solution notes
This notebook will hold my attempts at problems from *Elements of Programming Interviews* in Python. For each problem, record
* Brute force solution code
* Optimized solution code
* Book solution code
* Comments
    1. How does my solution compare with the book solution?
    2. What went right in my attempt?
    3. What went wrong in my attempt
    4. Do I need to learn new data structure or algorithms?

## Arrays 

### Buy and sell a stock once 
Given the price of a stock over a range of days, find the maximum profit one could make by buying the stock, then selling it on a future day. There is no need to buy if it is impossible to make a profit.

#### Attempt 1
It was helpful to think of the prices as elevation, with the array as a linear trail consisting of a series of peaks and valleys. I am a climber who would like to know the greatest increase in elevation while traveling one direction along the array. I can teleport, but I don't know the elevation at a given point until I travel to that location. I am given two flags labeled `lo` and `hi`.  

I begin by travelling downhill until I reach a valley as a starting point. If I reach the end of the trail without finding a valley, it means I never increased elevation and should return `0`. After placing `lo` at my starting point, I travel along the entire trail to find the highest peak and place `hi` at this location. I then move `lo` forward to the lowest valley before `hi` and record the elevation gain from `lo` to `hi` in my notebook. From this point, there could be a larger elevation change after `hi`, but not before it. I pick up my flags and repeat my steps on the remaining section of the trail, marking down the new elevation change. I will eventually reach the end of the trail, at which point I return the greatest elevation change I recorded in my notebook.

Consider a trail with $n$ points. The fastest time is if there is no elevation gain, in which case it takes $n-1$ steps to reach the end of the trail. If the highest peak is at the very end of the trail, the slowest time will be if the lowest valley is just before the highest peak at $n-2$. It will take $n-1$ steps to find the peak and $n-2$ steps to find the trail, or $2n-3$ steps in total. The worst case time will be $\sum_{i=1}^{m}(2l_i - 3)$, where $m$ is the number of segments and $l_i$ is the length of the $i^{th}$ segment. Thus, the time complexity is $O(n)$.

In [3]:
def buy_and_sell_stock_once(prices):
    n = len(prices)
    max_profit = 0
    lo = 0
    while lo < n-1:
        # Move lo to valley as starting point
        while prices[lo+1] <= prices[lo]:
            lo += 1
            if lo == n-1:
                return max_profit
        # Move hi to highest peak
        hi = lo+1
        for i in range(lo+1, n):
            if prices[i] > prices[hi]:
                hi = i
        # Move lo to lowest valley before hi 
        for i in range(lo+1, hi):
            if prices[i] < prices[lo]:
                lo = i
        # Record elevation gain
        local_profit = prices[hi] - prices[lo]
        max_profit = max(max_profit, local_profit)
        # Check next sub-array
        lo = hi + 1
    return max_profit

#### Book solution
Iterate through the array, remembering the lowest price so far. On day i, compute the profit we would make by buying at the lowest price seen so far and selling at the current price `prices[i]`.  

In [4]:
def buy_and_sell_stock_once(prices):
    min_price_so_far, max_profit = float('inf'), 0.0
    for price in prices:
        max_profit_sell_today = price - min_price_so_far
        max_profit = max(max_profit, max_profit_sell_today)
        min_price_so_far = min(min_price_so_far, price)
    return max_profit

#### Comments
Although my solution works and runs in about half the time as the book solution, it is hard to read on its own and took a while to reach. I could have reached the book solution quickly if I spent more time thinking before I wrote down any code.

1. My solution actually runs 2x faster than the book, but is much less compact and readable.
2. I solved the problem with brute force method quickly, and ended up with a working faster solution.
3. I started coding too quickly and missed the more compact solution. Thus the total time I took to solve the problem was much too long. 
4. No new DS or algorithms to learn.

### Buy and sell a stock twice
Given the price of a stock over a range of days, find the maximum profit one could make by buying and selling a stock twice, with the second buy on a day after the first sell. There is no need to buy if it is impossible to make a profit.

#### Attempt 1
Run the one buy/sell solution on left and right part of the array. It will be $O(n^2)$.

In [6]:
def buy_and_sell_stock_twice(prices):
    max_profit = 0
    for i in range(len(prices)):
        profit = buy_and_sell_stock_once(prices[:i]) + buy_and_sell_stock_once(prices[i:])
        max_profit = max(max_profit, profit)
    return max_profit

#### Attempt 2
I read the description of the solution but didn't copy; I tried to implement it myself. It basically goes through each day and looks at the maximum profit one could make by buying on a previous day selling today, as well as buying today and selling on a future day. The sum of these values is the maximum profit on a given day. The time complexity is $O(n)$ and the space complexity is $O(n)$.

In [5]:
def buy_and_sell_stock_twice(prices):
    min_so_far, max_so_far = float('inf'), float('-inf')
    n = len(prices)
    L, R = [0] * n, [0] * n
    for i in range(1, n):
        j = n-1-i
        min_so_far = min(min_so_far, prices[i-1])
        max_so_far = max(max_so_far, prices[j+1])
        L[i] = max(L[i-1], prices[i] - min_so_far)
        R[j] = max(R[j+1], max_so_far - prices[j])
    return max([sell+buy for sell,buy in zip(L, R)])

#### Book solution

In [10]:
def buy_and_sell_stock_twice(prices):

    max_total_profit, min_price_so_far = 0.0, float('inf')
    first_buy_sell_profits = [0.0] * len(prices)
    # Forward phase. For each day, we record maximum profit if we sell on that
    # day.
    for i, price in enumerate(prices):
        min_price_so_far = min(min_price_so_far, price)
        max_total_profit = max(max_total_profit, price - min_price_so_far)
        first_buy_sell_profits[i] = max_total_profit

    # Backward phase. For each day, find the maximum profit if we make the
    # second buy on that day.
    max_price_so_far = float('-inf')
    for i, price in reversed(list(enumerate(prices[1:], 1))):
        max_price_so_far = max(max_price_so_far, price)
        max_total_profit = max(
            max_total_profit,
            max_price_so_far - price + first_buy_sell_profits[i])
    return max_total_profit


def buy_and_sell_stock_twice_constant_space(prices):
    min_prices, max_profits = [float('inf')] * 2, [0] * 2
    for price in prices:
        for i in reversed(range(2)):
            max_profits[i] = max(max_profits[i], price - min_prices[i])
            min_prices[i] = min(min_prices[i],
                                price - (0 if i == 0 else max_profits[i - 1]))
    return max_profits[-1]

#### Comments
1. The book solution uses enumerate and is prettier. I guess the variable names are also more meaningful.
2. I got the $O(n^2)$ solution right away. Once I read the approach to the $O(n)$ solution, I was able to implement it myself.
3. I had trouble coming up with the $O(n)$ solution. 
4. Nothing new to learn, but be comfortable using `enumerate` and `reverse` to iterate backwards through lists.

### Rearrange and array 
Rearrange `A` so that `A[0] <= A[1] >= A[2] <= A[3] ...`. 

#### Attempt 1
Sort the array, then swap every other pair. $O(n\log{n})$ to sort.

In [3]:
def rearrange(A):
    A.sort()
    for i in range(1, len(A)-1, 2):
        A[i], A[i+1] = A[i+1], A[i]

#### Attempt 2
After getting hint from the book, I realized I can just sort locally.

In [8]:
def rearrange(A):
    for i in range(len(A)-1): 
        if (i % 2 and A[i] > A[i+1]) or (not i % 2 and A[i] < A[i+1]):
            A[i], A[i+1] = A[i+1], A[i]

#### Book solution
Same method as attempt 2

In [7]:
def rearrange(A):
    for i in range(len(A)):
        A[i:i+2] = sorted(A[i:i+2], reverse=i%2)

#### Comments
1. My solution is harder to read than the book solution.
2. I got an initial solution down quickly.
3. I got hung up on finding the median, which is not super easy. This distracted me from the other $O(n)$ solution.
4. Nothing new to learn here.

### Enumerate all primes to n 
Return all primes greater than 1 and less than or equal to $n$.

#### Method 1
I've done this problem before. Starting from i=2, run through the numbers and eliminate all multiples of i. Then move to the next number that's not crossed out and repeat. Do this until i >= $\sqrt{n}$. Here is the book's method. The next iteration does some optimization, but I won't list it here.

In [25]:
def generate_primes(n):
    primes = []
    is_prime = [False, False] + [True] * (n-1)
    for p in range(2, n+1):
        if is_prime[p]:
            primes.append(p)
            # Remove all multiples of p
            for i in range(p, n+1, p):
                is_prime[i] = False
    return primes

### Permute the elements of an array
$P$ is an array that tells how to permute an array. Example: $A = [a, b, c, d]$ and $P = [0, 2, 1, 3]$ would change $A$ to $[a, c, b, d]$. Write a function that applies $P$ to $A$.

#### Method 1
$O(n)$ time, $O(n)$ space.

In [30]:
def apply_permutation(perm, A):
    B = [0] * len(A)
    for i in range(len(A)):
        B[perm[i]] = A[i]
    A[:] = B

#### Method 2
$O(1)$ space, and I think $O(n)$ time. This method orders $P$ at the same time as $A$. It matches the book solution.

In [58]:
def apply_permutation(perm, A):
    for i in range(len(A)):
        while perm[i] != i:
            A[perm[i]], A[i] = A[i], A[perm[i]]
            perm[perm[i]], perm[i] = perm[i], perm[perm[i]]

#### Comments
1. My solution matches the solution in the code, but in the older book I have there is a much more complex solution.
2. This was a pretty easy problem, but I figured out the easy and the optimized solution.
3. I probably took too long.
4. Nothing new to learn.

### Compute the next permutation 

In [20]:
def next_permutation(perm):
    
    def swap(array, i, j):
        array[i], array[j] = array[j], array[i]

    # Check for short arrays
    n = len(perm)
    if n < 2:
        return perm

    perm[-2:] = sorted(perm[-2:, reverse=True])

    else:
        # Find anti-sorted section of perm
        i = n-2
        while perm[i] >= perm[i+1]:
            i -= 1
            if i < 0:
                return []
        # Find element in perm[i+1] which is next largest after perm[i]
        i_next_largest = i+1
        for j in range(i+2, n):
            if perm[j] > perm[i] and perm[j] < perm[i_next_largest]:
                i_next_largest = j
        # Swap that element and sort the remaining array
        swap(perm, i, i_next_largest)
        perm[i+1:] = sorted(perm[i+1:])
    return perm

perm = [2,2]
next_perm = next_permutation(perm)
print(next_perm)

[]


Time to solve = 1 hour.